In [1]:
#SETUP CELL (RUN ONCE)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
#SETUP CELL (RUN ONCE)
df_Calls_untidy = pd.read_csv("../data/Calls_for_Service_2021.csv")
df_zones_untidy = pd.read_csv("../data/Police_Zones_data.csv")
df_Calls = df_Calls_untidy.drop(columns=['NOPD_Item',
                                         'Type',
                                         'InitialType',
                                         'MapX',
                                         'MapY',
                                         'Disposition',
                                         'Beat'])
#df_Calls = df_Calls.astype({'TimeCreate':'datetime64[ns]',
#                            'TimeDispatch':'datetime64[ns]',
#                            "TimeArrive":'datetime64[ns]',
#                            "TimeClosed":'datetime64[ns]'})


NOTES:::
Display heatmap tables



QUESTIONS:::
*   Most frequent crimes in specific areas?

*   Based on the value counts of each type of crime in each area can we generalize patterns like violent crime happening more in one area, theft in another, etc?

*   What are the most frequest crimes by time of day in conjunction with a specific area?

In [4]:
#using df_Calls, find the most common crimes for each police district
#here lets make tables and stuff by grouping by zones, type of incidents, and other things that answer our question
#we should be good after this and then in our plan we can say we will be making a map of the data we find and then graphs and such of data stuff than cant be mapped

#Get 30 most frequent crimes
new_df = df_Calls.TypeText.value_counts()
top_Call_df = df_Calls[df_Calls.TypeText.isin(new_df[0:29].index)]



type_by_district = top_Call_df[['PoliceDistrict','TypeText']].pivot_table(index=['PoliceDistrict'],columns=['TypeText'],aggfunc=np.count_nonzero)
type_by_zipcode = top_Call_df[['Zip','TypeText']].pivot_table(index=['Zip'],columns=['TypeText'],aggfunc=np.count_nonzero)
#'''df_Calls_crossTab = pd.crosstab(df_Calls['PoliceDistrict'],df_Calls['TypeText'])
#display((df_Calls_crossTab.T / df_Calls_crossTab.T.sum()).T.plot(kind='bar',stacked=True, legend=False))
#type_by_district.plot(kind='bar', stacked=True, legend=False) #need to convert to marginal distribution for comparison'''

type_by_district.T[0].max()
df_Calls_marg_district = type_by_district.T / type_by_district.T.sum()
df_Calls_marg_zip = type_by_zipcode.T / type_by_zipcode.T.sum()
pop_district = type_by_district.T.idxmax()
pop_zip = type_by_zipcode.T.idxmax()
type_by_district_with_freq =  pd.merge(type_by_district, pop_district.rename('Most Frequent'), 'outer',on='PoliceDistrict')
type_by_zipcode_with_freq = pd.merge(type_by_zipcode, pop_zip.rename('Most Frequent'), 'outer',on='Zip')






Another model we are planning on making is analyzing the cross between the type of crime/ frequency to income and other related fields by zipcodes avaible in the 911 call data set. After analyzing the crossing of data, we hope to find a line or certain characteristics that make some zipcodes more likely to be involved in certain types of crimes. With this "line" or characteristcs, we can then predict the likelihood of a zipcode being involved in a certain type of crime, and be able to tell when zipcodes are creeping towards that line/ ways to escape and improve.

In [32]:
'''import census import Census
c = Census("6ef3d839845c6500b67b00a07c30c0d61a224218")
variable = '''
import requests
import numpy as np
from bs4 import BeautifulSoup as bf
import requests_html as reqHtml
zip_range = df_Calls.Zip.unique()

baseUrl = "https://data.census.gov/cedsci/all?q=ZCTA5"
space_ele = "%20"
income = []
health = []
employment =[]
education = []

requ_Variables = [("Income%20and%20Poverty",income),("%20Health",health),("%20employment",employment),("%20Education",education)]
browser = reqHtml.AsyncHTMLSession()
for zipcodeBase in zip_range:
    for requ_Variable in requ_Variables:
            info = browser.get(baseUrl+space_ele+zipcodeBase+space_ele+requ_Variable[0])  
            print(info)
            #info.html.render()
            parse_HTML = info.page_source
            print(parse_HTML)
            soup = bf(parse_HTML,"html.parser")
            result = soup.find("div",class_="aqua-card-title-text")
            result = result.text
            requ_Variable[1].append(result)
print(income)
print(health)
print(employment)
print(education)



Task exception was never retrieved
future: <Task finished name='Task-7' coro=<get_data() done, defined at C:\WINDOWS\TEMP/ipykernel_14452/2246762983.py:17> exception=AttributeError("'_asyncio.Future' object has no attribute 'text'")>
Traceback (most recent call last):
  File "C:\WINDOWS\TEMP/ipykernel_14452/2246762983.py", line 18, in get_data
    return await response.text()
AttributeError: '_asyncio.Future' object has no attribute 'text'


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at C:\Users\MSend\anaconda3\envs\Capstone\lib\asyncio\futures.py:384]>


AttributeError: '_asyncio.Future' object has no attribute 'page_source'

In [11]:
print(df_Calls.Zip.unique())
income = [96844,66499,np.nan,52312,27026,30806,30264,48919,18148,39827,
          49439,39506,27190,53946,36414,48919,46526,np.nan,36950,np.nan] #income from 2019
health = [3.8,7.2,np.nan,8.9,9.0,10.8,13.6,12.7,14.5,11.6,
          12.1,9.2,8.6,8.1,7.5,7.6,7.9,np.nan,11.7,np.nan] #percent without health care coverage from 2019
employment =[]
education = []


['70124' '70115' 'None' '70131' '70126' '70114' '70117' '70116' '70113'
 '70119' '70112' '70122' '70127' '70130' '70128' '70118' '70125' nan
 '70129' '70148']


In [ ]:
df_Calls.TypeText.unique()
violent_type =  df_Calls.TypeText.isin(['BURGLAR ALARM, SILENT', 
       'DISCHARGING FIREARM', 'AUTO ACCIDENT WITH INJURY', 'AGGRAVATED CRIMINAL DAMAGE',
       'HIT & RUN WITH INJURIES', 'FIGHT',
       'DOMESTIC DISTURBANCE', 'SIMPLE BATTERY', 'HIT & RUN', 'MEDICAL',
       'HOLD UP ALARM', 'EXTORTION (THREATS)',
       'BURGLAR ALARM, SILENT, FAR NO RESPONSE', 'AGGRAVATED BATTERY DOMESTIC', 
       'ARMED ROBBERY WITH KNIFE', 'RECKLESS DRIVING', 'SIMPLE BATTERY DOMESTIC', 
       'MISDEMEANOR SEXUAL BATTERY', 'BURGLAR ALARM, LOCAL',
       'AGGRAVATED BATTERY BY SHOOTING', 'SIMPLE BURGLARY VEHICLE',
       'SIMPLE CRIMINAL DAMAGE',
       'SIMPLE BURGLARY', 
       'AGGRAVATED ASSAULT', 'VIOLATION OF PROTECTION ORDER', 'PROWLER',
       'RESIDENCE CHECK', 'AGGRAVATED BURGLARY DOMESTIC',
       'DRUG VIOLATIONS', 'VEHICLE NO-PURSUIT',
       'CRIMINAL DAMAGE DOMESTIC', 'MISSING ADULT', 'RESIDENCE BURGLARY',
       'MISSING JUVENILE', 'SIMPLE ROBBERY, PROPERTY SNATCHING',
       'UNCLASSIFIED DEATH', 'RECOVERY OF REPORTED STOLEN VEHICLE',
       'SIMPLE ASSAULT DOMESTIC', 'OBSCENITY, EXPOSING',
       'AGGRAVATED ASSAULT DOMESTIC', 'DIRECTED PATROL',
       'THEFT FROM EXTERIOR OF VEHICLE', 'PICKPOCKET',
       'AGGRAVATED BATTERY BY CUTTING', 'AUTO ACCIDENT CITY VEHICLE',
       'SIMPLE ROBBERY', 'FUGITIVE ATTACHMENT', 'ARMED ROBBERY WITH GUN',
       'AGGRAVATED BATTERY', 'THEFT BY FRAUD', 'ARMED ROBBERY',
       'BUSINESS BURGLARY', 'AUTO ACCIDENT FATALITY', 'FIRE',
       'SIMPLE RAPE', 'WALKING BEAT', 'SUICIDE ATTEMPT',
       'CRIMINAL MISCHIEF', 'SEXUAL BATTERY', 'RECOVERY OF VEHICLE',
       'AGGRAVATED RAPE UNFOUNDED BY SPECIAL VICTIMS OR CHILD ABUSE',
       'POSSESSION OF STOLEN PROPERTY', 'EXTORTION (THREATS) DOMESTIC',
       'CARJACKING- NO WEAPON', 'VEHICLE PURSUIT', 'FORGERY',
       'SIMPLE BURGLARY DOMESTIC', 'SEXTING', 'CARJACKING',
       'MEDICAL SEXUAL ASSAULT KIT PROCESSING',
       'AUTO ACCIDENT POLICE VEHICLE', 'DIRECTED TRAFFIC ENFORCEMENT',
       'ILLEGAL CARRYING OF WEAPON- GUN',
       'SIMULTANEOS STOLEN/RECOVERY VEHICLE', 'ABANDONED BOAT',
       'HIT & RUN CITY VEHICLE', 'AGGRAVATED RAPE', 'BICYCLE THEFT',
       'INDECENT BEHAVIOR', 'ILLEGAL CARRYING OF WEAPON',
       'SIMPLE KIDNAPPING', 'QUALITY OF LIFE ISSUE', 'CRUELTY TO ANIMALS',
       'HOMICIDE BY SHOOTING', 'HOMELESS', 'TRUANT VIOLATION',
       'ISSUING WORTHLESS CHECKS', 'SIMPLE ASSAULT', 'FLOOD EVENT',
       'CAD TEST', 'PROTEST', 'DISPERSE SUBJECTS',
       'SEX OFFENDER REGISTRATION CHECK', 'TRAFFIC ATTACHMENT',
       'NEGLIENT INJURY', 'HOMICIDE DOMESTIC', 'AGGRAVATED BURGLARY',
       'ASSET SEIZURE', 'THEFT BY EMBEZZLEMENT', 'DESECRATION OF GRAVES',
       'SUSPICIOUS PACKAGE', 'JUVENILE ATTACHMENT', 'VIDEO VOYEURISM',
       'SIMPLE ARSON', 'MEET AN OFFICER', 'CONTRIBUTING TO DELINQUENCY',
       'CURFEW VIOLATION', 'SUICIDE', 'MEDICAL - NALOXONE',
       'HIT & RUN FATALITY', 'BOMB SCARE',
       'SIMPLE RAPE UNFOUNDED BY SPECIAL VICTIMS OR CHILD ABUSE',
       'AGGRAVATED KIDNAPPING', 'DRUNK', 'HOMICIDE',
       'HIT & RUN POLICE VEHICLE', 'OFFICER NEEDS ASSISTANCE',
       'HOMICIDE BY CUTTING', 'PROSTITUTION',
       'OFFICER NEEDS ASSISTANCE, LIFE IN DANGER', 'BLIGHTED PROPERTY',
       'ILLEGAL CARRYING OF WEAPON- KNIFE', 'ICING ON ROADS',
       'SOLICITING FOR PROSTITUTION', 'BARRICADE ERECTED',
       'AGGRAVATED ARSON', 'ATTACHMENT', 'EXPLOSION',
       'CRIME AGAINST NATURE', 'FLOODED VEHICLE', 'TRAFFIC CONGESTION',
       'EMS UNIT NEEDS ASSISTANCE', 'CRIMINAL MISCHIEF DOMESTIC',
       'AGGRAVATED RAPE MALE VICTIM', 'PEEPING TOM', 'AIRPLANE CRASH',
       'HOSTAGE SITUATION', 'SIMPLE ARSON DOMESTIC', 'SIMPLE ESCAPE',
       'IMPERSONATING AN OFFICER', 'AGGRAVATED ESCAPE',
       'REFERRAL TO MHSD', 'TROOP N AREA - BUSINESS - RESIDENCE CHECK',
       'SIMPLE RAPE MALE VICTIM', 'PARADE ITEM NUMBER'])#true and false